In [1]:
#pip install datasets transformers

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
import math 
import torch

device = torch.device("cpu")
if torch.cuda.is_available():
 device = torch.device("cuda")
device

c:\Users\Hallgrimur\anaconda3\lib\site-packages\torch\cuda\__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [3]:
#Load data
dataset = load_dataset("copenlu/answerable_tydiqa")
dataset

Using custom data configuration copenlu--nlp_course_tydiqa-cceecfb5416d988a
Found cached dataset parquet (C:/Users/Hallgrimur/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-cceecfb5416d988a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
})

In [4]:
#split data 
train_set = dataset["train"]
validation_set = dataset["validation"]

train_set = train_set.to_pandas()
validation_set = validation_set.to_pandas()
validation_set

,question_text,document_title,language,annotations,document_plaintext,document_url
0,อยุธยามีกี่อำเภอ ?,จังหวัดพระนครศรีอยุธยา,thai,"{'answer_start': [42], 'answer_text': ['16']}",\nปัจจุบันจังหวัดพระนครศรีอยุธยาประกอบด้วย 16 ...,https://th.wikipedia.org/wiki/%E0%B8%88%E0%B8%...
1,كم عدد مرات فوز الأوروغواي ببطولة كاس العالم ل...,قائمة نهائيات كأس العالم,arabic,"{'answer_start': [394], 'answer_text': ['بطولت...",أقيمت البطولة 21 مرة، شارك في النهائيات 78 دول...,https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%...
2,Siapakah yang menemuka benua Amerika ?,Kristoforus Kolumbus,indonesian,"{'answer_start': [193], 'answer_text': ['orang...",Kolumbus bukanlah orang pertama yang tiba di A...,https://id.wikipedia.org/wiki/Kristoforus%20Ko...
3,化学兵器禁止条約はどこで採択された？,化学兵器禁止条約,japanese,"{'answer_start': [11], 'answer_text': ['パリ']}",1993年1月13日にパリにおいて署名がなされ、1997年4月29日に発効した[1]。実効的...,https://ja.wikipedia.org/wiki/%E5%8C%96%E5%AD%...
4,ใครเป็นผู้ก่อการ ใน เหตุโจมตีในนอร์เวย์ พ.ศ. 2...,เหตุโจมตีในนอร์เวย์ พ.ศ. 2554,thai,"{'answer_start': [400], 'answer_text': ['แอนเด...",เหตุโจมตีครั้งที่สองนั้นเกิดขึ้นในราวสองชั่วโม...,https://th.wikipedia.org/wiki/%E0%B9%80%E0%B8%...
...,...,...,...,...,...,...
13320,월드컵이 처음 개최된 나라는 어디인가?,FIFA 월드컵,korean,"{'answer_start': [-1], 'answer_text': ['']}",브라질의 펠레는 FIFA 월드컵에 출전한 모든 선수들 중 최고로 월드컵 기록이 좋은...,https://ko.wikipedia.org/wiki/FIFA%20%EC%9B%94...
13321,Berapakah berat Ikan pari manta yag terbesar?,Ikan pari manta,indonesian,"{'answer_start': [-1], 'answer_text': ['']}","Ciri khas manta adalah sepasang ""tanduk"" di de...",https://id.wikipedia.org/wiki/Ikan%20pari%20manta
13322,Когда впервые начали применять компьютерную гр...,CGI (графика),russian,"{'answer_start': [-1], 'answer_text': ['']}",Созданием движущихся изображений занимается ко...,https://ru.wikipedia.org/wiki/CGI%20%28%D0%B3%...
13323,หม่อมราชวงศ์สุขุมพันธุ์ บริพัตร เกิดเมื่อไหร่?,หม่อมราชวงศ์สุขุมพันธุ์ บริพัตร,thai,"{'answer_start': [-1], 'answer_text': ['']}",ลำดับหม่อมราชวงศ์สุขุมพันธุ์ บริพัตร\n\nพระบาท...,https://th.wikipedia.org/wiki/%E0%B8%AB%E0%B8%...


In [5]:
chosen_language = ['english', 'japanese', 'finnish']

train_set = train_set[train_set['language'].isin(chosen_language)]
validation_set = validation_set[validation_set['language'].isin(chosen_language)]

validation_set

,question_text,document_title,language,annotations,document_plaintext,document_url
3,化学兵器禁止条約はどこで採択された？,化学兵器禁止条約,japanese,"{'answer_start': [11], 'answer_text': ['パリ']}",1993年1月13日にパリにおいて署名がなされ、1997年4月29日に発効した[1]。実効的...,https://ja.wikipedia.org/wiki/%E5%8C%96%E5%AD%...
9,オリヴィア・デ・ハヴィランドが生まれたのはいつ,オリヴィア・デ・ハヴィランド,japanese,"{'answer_start': [46], 'answer_text': ['1916年7...","\nオリヴィア・デ・ハヴィランド（Dame Olivia De Havilland, DBE...",https://ja.wikipedia.org/wiki/%E3%82%AA%E3%83%...
11,Kauanko lasia on valmistettu?,Lasi,finnish,"{'answer_start': [160], 'answer_text': ['noin ...",Vanhin tunnettu lasilaatu on alkali­kalkki­las...,https://fi.wikipedia.org/wiki/Lasi
14,Mikä on Ponzi-huijaus?,Ponzi-huijaus,finnish,"{'answer_start': [39], 'answer_text': ['pyrami...",Ponzi-huijaus eli Ponzi-järjestelmä on pyramid...,https://fi.wikipedia.org/wiki/Ponzi-huijaus
20,Mikä oli Napoleonin sotien lopputulos?,Napoleonin sodat,finnish,"{'answer_start': [15], 'answer_text': ['Ranska...",Sotien jälkeen Ranska alistettiin kovilla rauh...,https://fi.wikipedia.org/wiki/Napoleonin%20sodat
...,...,...,...,...,...,...
13305,What is the most common first word by babies?,Vocabulary development,english,"{'answer_start': [-1], 'answer_text': ['']}","Social pragmatic theories, also in contrast to...",https://en.wikipedia.org/wiki/Vocabulary%20dev...
13307,Kuinka kauan valtiopäivät kestää?,Suomen valtiopäivät,finnish,"{'answer_start': [-1], 'answer_text': ['']}",\nEnnen vuotta 1906 kokoontuivat säätyvaltiopä...,https://fi.wikipedia.org/wiki/Suomen%20valtiop...
13315,アイスランド共和国の首都はどこですか？,アイスランド,japanese,"{'answer_start': [-1], 'answer_text': ['']}",こうした危機を乗り切るため、アイスランド中央銀行は8日にロシアから40億ユーロの緊急融資を受...,https://ja.wikipedia.org/wiki/%E3%82%A2%E3%82%...
13317,Milloin käytiin Persianlahden sota?,Persianlahden sota,finnish,"{'answer_start': [-1], 'answer_text': ['']}",Ilmaiskuilla oli erityisen tuhoisa vaikutus Ir...,https://fi.wikipedia.org/wiki/Persianlahden%20...


In [6]:
def get_answerability(annotations):
    answerable = []
    for annot in annotations:
        if -1 in annot['answer_start']:
            answerable.append(0)
        else:
            answerable.append(1)
    return answerable

train_set['answerable'] = get_answerability(train_set['annotations'].tolist())
validation_set['answerable'] = get_answerability(validation_set['annotations'].tolist())

train_set.columns

C:\Users\Hallgrimur\AppData\Local\Temp\ipykernel_13932\2290379095.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_set['answerable'] = get_answerability(validation_set['annotations'].tolist())


Index(['question_text', 'document_title', 'language', 'annotations',
       'document_plaintext', 'document_url', 'answerable'],
      dtype='object')

In [7]:
from datasets import DatasetDict, Dataset

def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += [questions[index] + "\n" + paragraphs[index]]
        
    return training_data

def make_df(train_set, validation_set):

  d_q = train_set['question_text'].tolist()
  d_p = train_set['document_plaintext'].tolist()
  training_data = question_parag_combine(d_q,d_p)
  training_labels = train_set['answerable'].tolist()

  d_q = validation_set['question_text'].tolist()
  d_p = validation_set['document_plaintext'].tolist()
  validation_data = question_parag_combine(d_q,d_p)
  validation_labels = validation_set['answerable'].tolist()



  data_set = {}
  sets = [['train',training_data, training_labels], ['val', validation_data, validation_labels]]
  for meta in sets:
      data_set[meta[0]] = {}
      data_set[meta[0]]['text'] = []
      data_set[meta[0]]['labels'] = []
      
      for ind, text in enumerate(meta[1]):
          data_set[meta[0]]['text'].append(text)
          data_set[meta[0]]['labels'].append(meta[2][ind])

          

          
  data_set = DatasetDict({'train':Dataset.from_dict(data_set['train']),
                          'valid':Dataset.from_dict(data_set['val'])\
                        })
  return data_set

data_set = make_df(train_set, validation_set)
data_set

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 29868
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 3712
    })
})

In [28]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    keys = ['attention_mask', 'input_ids', 'token_type_ids']
    concatenated_examples = {k: sum(examples[k], []) for k in keys}
    total_length = len(concatenated_examples[list(keys)[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # this is needed as the used dataset is a subclass of ClassificationDataset, which requires label as a field...
    result["label"] = result["input_ids"].copy()
    result["labels"] = result["input_ids"].copy()
    return result

In [46]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, T5Tokenizer, GPT2LMHeadModel, GPT2Tokenizer, GPT2Model

#define multilingual model
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

#tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\Hallgrimur/.cache\huggingface\transformers\6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embeddi

In [47]:
#tokenize data
tokenized_datasets = data_set.map(tokenize_function, batched=True, num_proc=1, remove_columns=['text'])

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [48]:
tokenized_datasets_lm = tokenized_datasets.map(group_texts, batched=True, batch_size=512, num_proc=1)
tokenized_datasets_lm = tokenized_datasets_lm.remove_columns(['label'])

  0%|          | 0/59 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [49]:
tokenized_datasets_lm

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 119472
    })
    valid: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14848
    })
})

In [50]:
#training arguments

training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    #max_steps=300
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_lm["train"],
    eval_dataset=tokenized_datasets_lm["valid"],
)


In [52]:
trainer_result = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 14848
  Batch size = 8


  0%|          | 0/1856 [00:00<?, ?it/s]

KeyboardInterrupt: 